# <center>Загрузка данных (daily) S&P500(100)

Загрузка данных осуществляется с сайта https://finance.yahoo.com/

In [1]:
import os
os.chdir("..")

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests
import pandas as pd
import time
import json
import shutil

In [3]:
import urllib
from bs4 import BeautifulSoup

In [4]:
path = 'prices'

## Функция загрузки данных

In [6]:
def load_data(ticker, path='prices'):
    filename = f'{ticker}.csv'
    path_to_file = os.path.join(path, filename)
    url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}'\
          f'?period1=947894400&period2=1600128000&interval=1d&events=history'
    r = requests.get(url)
    content = r.content.decode('UTF-8')
    with open(path_to_file, "w") as file:
        file.write(content)
#     print(f'\tUpdate {ticker} price history:', path_to_file)
    return path_to_file

#### testing

In [12]:
ticker = 'AAPL'
path_to_file = load_data(ticker)

In [13]:
data = pd.read_csv(path_to_file)

In [14]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-18,0.901786,0.946429,0.896763,0.928013,0.801933,459177600
1,2000-01-19,0.943080,0.970982,0.922991,0.951451,0.822186,597643200
2,2000-01-20,1.031250,1.084821,1.013393,1.013393,0.875712,1831132800
3,2000-01-21,1.020089,1.020089,0.983817,0.993862,0.858835,495924800
4,2000-01-24,0.968192,1.006696,0.938616,0.948661,0.819775,440876800


In [15]:
data.shape

(5198, 7)

## Формируем список акций индекса S&P500

Разбор [страницы Википедии](http://en.wikipedia.org/wiki/List_of_S%26P_500_companies) для получения списка акций, входящих в индекс S&P 500.    
Сгруппируем по секторам.

In [16]:
url = "http://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = urllib.request.urlopen(url)
html = response.read()

In [17]:
soup = BeautifulSoup(html)

# Найдём таблицу:
table = soup.find('table', {'class': 'wikitable sortable'})

# Словарь для тикеров, сгруппированных по секторам экономики:
sp500_tickers_by_sectors = dict()

for r in table.findAll('tr'):  # Цикл по строкам таблицы.
    col = r.findAll('td')      # Колонки таблицы.
    if len(col)>0:
        # В 4-й колонке таблицы - название сектора экономики:
        sector = str(col[3].string.strip()).lower().replace(' ', '_')

        # В 1-й колонке таблицы - тикер акции:
        ticker = str(col[0].find('a').string.strip())

        # Если пока нет такого сектора, добавляем к словарю новый список для тикеров:
        if sector not in sp500_tickers_by_sectors:
            sp500_tickers_by_sectors[sector] = list()
            
        # Добавляем тикер акции в список соответствующего сектора:
        sp500_tickers_by_sectors[sector].append(ticker)

In [18]:
import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(sp500_tickers_by_sectors)

{'communication_services': ['ATVI',
                            'GOOGL',
                            'GOOG',
                            'T',
                            'CTL',
                            'CHTR',
                            'CMCSA',
                            'DISCA',
                            'DISCK',
                            'DISH',
                            'EA',
                            'FB',
                            'FOXA',
                            'FOX',
                            'IPG',
                            'LYV',
                            'NFLX',
                            'NWSA',
                            'NWS',
                            'OMC',
                            'TMUS',
                            'TTWO',
                            'TWTR',
                            'VZ',
                            'VIAC',
                            'DIS'],
 'consumer_discretionary': ['AAP',
                            'AMZN',
    

In [19]:
print(sp500_tickers_by_sectors)

{'industrials': ['MMM', 'ALK', 'ALLE', 'AAL', 'AME', 'AOS', 'BA', 'CHRW', 'CARR', 'CAT', 'CTAS', 'CPRT', 'CSX', 'CMI', 'DE', 'DAL', 'DOV', 'ETN', 'EMR', 'EFX', 'EXPD', 'FAST', 'FDX', 'FLS', 'FTV', 'FBHS', 'GD', 'GE', 'GWW', 'HON', 'HWM', 'HII', 'IEX', 'INFO', 'ITW', 'IR', 'J', 'JBHT', 'JCI', 'KSU', 'LHX', 'LMT', 'MAS', 'NLSN', 'NSC', 'NOC', 'ODFL', 'OTIS', 'PCAR', 'PH', 'PNR', 'PWR', 'RTX', 'RSG', 'RHI', 'ROK', 'ROL', 'ROP', 'SNA', 'LUV', 'SWK', 'TDY', 'TXT', 'TT', 'TDG', 'UNP', 'UAL', 'UPS', 'URI', 'VRSK', 'WAB', 'WM', 'XYL'], 'health_care': ['ABT', 'ABBV', 'ABMD', 'A', 'ALXN', 'ALGN', 'ABC', 'AMGN', 'ANTM', 'BAX', 'BDX', 'BIO', 'BIIB', 'BSX', 'BMY', 'CAH', 'CNC', 'CERN', 'CI', 'COO', 'CVS', 'DHR', 'DVA', 'XRAY', 'DXCM', 'EW', 'GILD', 'HCA', 'HSIC', 'HOLX', 'HUM', 'IDXX', 'ILMN', 'INCY', 'ISRG', 'IQV', 'JNJ', 'LH', 'LLY', 'MCK', 'MDT', 'MRK', 'MTD', 'MYL', 'PKI', 'PRGO', 'PFE', 'DGX', 'REGN', 'RMD', 'STE', 'SYK', 'TFX', 'TMO', 'UNH', 'UHS', 'VAR', 'VRTX', 'WAT', 'WST', 'ZBH', 'ZTS'], 

In [20]:
sp500_tickers_by_sectors.keys()

dict_keys(['industrials', 'health_care', 'information_technology', 'communication_services', 'consumer_discretionary', 'utilities', 'financials', 'materials', 'real_estate', 'consumer_staples', 'energy'])

In [21]:
with open('sp500_tickers_by_sectors.json', 'w') as f:
    json.dump(sp500_tickers_by_sectors, f)

In [23]:
# with open("sp500_tickers_by_sectors.json", "r") as f:
#     sp500_tickers_by_sectors_ = json.load(f)
# sp500_tickers_by_sectors_

Объединим все тикеры для удобства скачивания в один общий список

In [24]:
sp500_tickers = []
for sector in sp500_tickers_by_sectors.keys():
    sp500_tickers.extend(sp500_tickers_by_sectors[sector])

In [25]:
print(sorted(sp500_tickers))

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BRK.B', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CDW', 'CE', 'CERN', 'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'COTY', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTL', 'CTSH', 'CTVA', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DL

In [26]:
len(sp500_tickers)

505

In [27]:
with open('sp500_tickers.json', 'w') as f:
    json.dump(sp500_tickers, f)

## Скачиваем котировки акций согласно списку для индекса **S&P500**

Из-за некоторых ограничений на скачивание с сайта https://finance.yahoo.com/ данные скачиваем в несколько этапов и с периодической минутной задержкой

In [30]:
%%time
for num, ticker in enumerate(sp500_tickers):
    load_data(ticker)
    if num % 5 == 0:
        time.sleep(60)

Wall time: 1h 44min 46s


докачиваем нестандарт (BRK-B вместо BRK.B и BF-B вместо BF.B и затем переименуем в именах "-" на ".")

In [37]:
load_data("BRK-B")

'prices\\BRK-B.csv'

In [38]:
load_data("BF-B")

'prices\\BF-B.csv'

## Формируем список акций индекса S&P100

Разбор [страницы Википедии](https://en.wikipedia.org/wiki/S%26P_100) для получения списка акций, входящих в индекс S&P100.    

In [31]:
url = "https://en.wikipedia.org/wiki/S%26P_100"
response = urllib.request.urlopen(url)
html = response.read()

In [32]:
soup = BeautifulSoup(html)

# Найдём таблицу:
table = soup.find('table', {'class': 'wikitable sortable'})

# Словарь для тикеров, сгруппированных по секторам экономики:
sp100_tickers = list()

for r in table.findAll('tr'):  # Цикл по строкам таблицы.
    col = r.findAll('td')      # Колонки таблицы.
    if len(col)>0:

        # В 1-й колонке таблицы - тикер акции:
        ticker = str(col[0].string.strip())
            
        # Добавляем тикер акции в список:
        sp100_tickers.append(ticker)

In [33]:
print(sp100_tickers)

['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'ALL', 'AMGN', 'AMT', 'AMZN', 'AXP', 'BA', 'BAC', 'BIIB', 'BK', 'BKNG', 'BLK', 'BMY', 'BRK.B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FB', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD', 'MDLZ', 'MDT', 'MET', 'MMM', 'MO', 'MRK', 'MS', 'MSFT', 'NEE', 'NFLX', 'NKE', 'NVDA', 'ORCL', 'OXY', 'PEP', 'PFE', 'PG', 'PM', 'PYPL', 'QCOM', 'RTX', 'SBUX', 'SLB', 'SO', 'SPG', 'T', 'TGT', 'TMO', 'TXN', 'UNH', 'UNP', 'UPS', 'USB', 'V', 'VZ', 'WBA', 'WFC', 'WMT', 'XOM']


In [34]:
len(sp100_tickers)

101

In [35]:
with open('sp100_tickers.json', 'w') as f:
    json.dump(sp100_tickers, f)

скопируем файлы акций **S&P100** в отдельную директорию, чтобы изучить объемы файлов и посмотреть, по каким акциям отсутствуют дальние котировки

In [36]:
for ticker in sp100_tickers:
    shutil.copy(os.path.join('prices', f'{ticker}.csv'), 'prices_sp100')

## Скачаем сами индексы S&P500 (тикер SPX) и S&P100 (тикер OEX)

Эти индексы скачаем вручную с сайта https://finance.yahoo.com/
- **S&P500** (тикер **SPX**) - со страницы [отсюда](https://finance.yahoo.com/quote/%5EGSPC/history?period1=914889600&period2=1600128000&interval=1d&filter=history&frequency=1d)   
- **S&P100** (тикер **OEX**) - со страницы [отсюда](https://finance.yahoo.com/quote/%5EOEX/history?period1=916358400&period2=1600128000&interval=1d&filter=history&frequency=1d)